In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import h5py

In [2]:
# Define paths to the data
ndvi_folder = 'C:\\Users\\Musae\\Documents\\GitHub-REPOs\\Senior-project_Doc\\Docs\\Array\\NDVI-Array'
ndmi_folder = 'C:\\Users\\Musae\\Documents\\GitHub-REPOs\\Senior-project_Doc\\Docs\\Array\\NDMI-Array'
climate_data_path = "C:\\Users\\Musae\\Documents\\GitHub-REPOs\\Senior-project_Doc\\monthly_averages_formatted.csv"

# Load climate data
climate_data = pd.read_csv(climate_data_path)

In [3]:
# Load pixel-wise NDVI and NDMI data from .npy files
# Assuming the files are sorted and aligned by date in the folder
ndvi_files = sorted([f for f in os.listdir(ndvi_folder) if f.endswith('.npy')])
ndmi_files = sorted([f for f in os.listdir(ndmi_folder) if f.endswith('.npy')])

In [4]:


# Load climate data
climate_data = pd.read_csv(climate_data_path)

# Load pixel-wise NDVI data from .npy files, assuming the files are sorted and aligned by date in the folder
ndvi_files = sorted([f for f in os.listdir(ndvi_folder) if f.endswith('.npy')])

# Initialize a Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Define batch size and prepare for batch processing
batch_size = 10  # Adjust based on your system's memory capacity
batch_features = []
batch_labels = []

# Process each NDVI file
for idx, ndvi_file in enumerate(ndvi_files):
    ndvi_path = os.path.join(ndvi_folder, ndvi_file)
    ndvi_data = np.load(ndvi_path)

    # Extract the date part from the NDVI filename and format it to match 'Year/Month'
    date_parts = ndvi_file.split('_')[-1].split('-')
    year_month = f"{date_parts[0]}/{date_parts[1]}"

    # Find matching climate data
    matching_climate_data = climate_data[climate_data['Year/Month'] == year_month]
    if not matching_climate_data.empty:
        temp = matching_climate_data['Temp Average'].iloc[0]
        precip = matching_climate_data['PRECTOTCORR Average'].iloc[0]

        # Flatten the NDVI data to handle each pixel as a separate data point
        ndvi_flat = ndvi_data.flatten()

        # Append features and labels to the batch
        for ndvi_val in ndvi_flat:
            batch_features.append([temp, precip])
            batch_labels.append(ndvi_val)

        # Process the batch when it reaches the batch size or end of files
        if (idx + 1) % batch_size == 0 or (idx + 1) == len(ndvi_files):
            # Convert batch to numpy arrays
            X_batch = np.array(batch_features)
            y_batch = np.array(batch_labels)

            # Split batch into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(X_batch, y_batch, test_size=0.2, random_state=42)
            
            # Train the model
            model.fit(X_train, y_train)

            # Clear the batch lists for the next batch
            batch_features = []
            batch_labels = []

    else:
        print(f"No matching climate data found for date: {year_month}")

# Final evaluation on the last test set
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"Final Root Mean Squared Error for pixel-wise NDVI prediction: {rmse}")


KeyboardInterrupt: 